In [1]:
!pip install --upgrade numpy
!pip install --upgrade scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 660.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505214 sha256

In [2]:
!pip uninstall -y numpy scikit-surprise
!pip install "numpy<2.0"
!pip install scikit-surprise

Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Successfully uninstalled numpy-2.2.5
Found existing installation: scikit-surprise 1.1.4
Uninstalling scikit-surprise-1.1.4:
  Successfully uninstalled scikit-surprise-1.1.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
  Using cached scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl


In [3]:
# import numpy as np
# print(np.__version__)
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.4 MB/s eta 0:00:00


In [4]:
# pip uninstall scikit-surprise numpy -y
!pip install numpy==1.23.5
!pip install scikit-surprise --no-binary :all:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have nu

In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "MONGO_URI"

client = MongoClient(uri, server_api=ServerApi("1"))

try:
    client.admin.command("ping")
    print("Connected successfully to the correct cluster!")
except Exception as e:
    print("Connection error:", e)


✅ Connected successfully to the correct cluster!


In [9]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from pymongo import MongoClient
import ast
import os
# from dotenv import load_dotenv

# load_dotenv()

# MongoDB Connection
# client = MongoClient(os.getenv("MONGO_URI"))

# Collections
provenance_coll = client["log_database"]["request_provenance_log"]
ratings_coll = client["movie_database"]["user_rate_data"]
users_coll = client["user_database"]["user_info"]

# Load Data
ratings = pd.DataFrame(list(ratings_coll.find({}, {"_id": 0})))
users = pd.DataFrame(list(users_coll.find({}, {"_id": 0, "user_id": 1, "gender": 1})))
provenance_logs = list(provenance_coll.find({"pipeline_type": "SVD"}))

ratings["user_id"] = ratings["user_id"].astype(str)
ratings["movie_id"] = ratings["movie_id"].astype(str)
users["user_id"] = users["user_id"].astype(str)

# Merge ratings with gender info
ratings_gender = ratings.merge(users, on="user_id", how="left")

# Train SVD Model
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings[["user_id", "movie_id", "score"]].rename(columns={"score": "rating"}), reader)
trainset = data.build_full_trainset()

model = SVD()
model.fit(trainset)

# Group predictions by gender
male_preds = []
female_preds = []

# Create dict for gender lookup
user_gender_map = dict(zip(users["user_id"], users["gender"]))

for log in provenance_logs:
    uid = str(log["user_id"])
    recommended = log.get("recommendation_results", [])
    ground_truth = ratings[(ratings["user_id"] == uid) & (ratings["movie_id"].isin(recommended))]

    if ground_truth.empty:
        continue

    testset = [(row["user_id"], row["movie_id"], row["score"]) for _, row in ground_truth.iterrows()]
    preds = [model.predict(u, i, r) for u, i, r in testset]

    gender = user_gender_map.get(uid)
    if gender == "M":
        male_preds.extend(preds)
    elif gender == "F":
        female_preds.extend(preds)

# Calculate RMSE
print("\nRMSE Results by Gender:")

rmse_male = None
rmse_female = None

if male_preds:
    rmse_male = accuracy.rmse(male_preds, verbose=True)
else:
    print("No male predictions available.")

if female_preds:
    rmse_female = accuracy.rmse(female_preds, verbose=True)
else:
    print("No female predictions available.")

# Print final values nicely
print("\n----------------------------------")
print(f"Final RMSE for Male Users:   {rmse_male:.4f}" if rmse_male else "No male RMSE computed.")
print(f"Final RMSE for Female Users: {rmse_female:.4f}" if rmse_female else "No female RMSE computed.")
print("----------------------------------")



RMSE Results by Gender:
RMSE: 0.3221
RMSE: 0.5102

----------------------------------
Final RMSE for Male Users:   0.3221
Final RMSE for Female Users: 0.5102
----------------------------------
